<a href="https://colab.research.google.com/github/farshidbalan/FluentPython/blob/master/Chapter16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coroutines

A coroutine is syntactically like a generator: just a function with the yield keyword in
its body. However, in a coroutine, yield usually appears on the right side of an expression (e.g., datum = yield), and it may or may not produce a value—if there is no expression after the yield keyword, the generator yields None. The coroutine may receive data from the caller, which uses .send(datum) instead of next(…) to feed the
coroutine. Usually, the caller pushes values into the coroutine.

It is even possible that no data goes in or out through the yield keyword. Regardless of
the flow of data, yield is a control flow device that can be used to implement cooperative
multitasking: each coroutine yields control to a central scheduler so that other coroutines can be activated.



## Basic Behavior of a Generator Used as a Coroutine

In [4]:
# Example 16-1. Simplest possible demonstration of coroutine in action
def simple_coroutine():
  print('-> coroutine started')  # A coroutine is defined as a generator function: with yield in its body.
  x = yield
  print('-> coroutine recieved:', x)  # yield is used in an expression; when the coroutine is designed just to receive data from the client it yields 
                                      # None—this is implicit because there is no expression to the right of the yield keyword.
  
my_coro = simple_coroutine()
my_coro

<generator object simple_coroutine at 0x7fa9c8097620>

In [5]:
next(my_coro)

-> coroutine started


In [6]:
my_coro.send(42) # This call makes the yield in the coroutine body evaluate to 42; now the coroutine resumes and runs until the next yield or termination.

-> coroutine recieved: 42


StopIteration: ignored

### Remark

A coroutine can be in one of four states. You can determine the current state using the
inspect.getgeneratorstate(…) function, which returns one of these strings:

- 'GEN_CREATED'
Waiting to start execution.

- 'GEN_RUNNING'
Currently being executed by the interpreter

- 'GEN_SUSPENDED'
Currently suspended at a yield expression.

- 'GEN_CLOSED'
Execution has completed

Because the argument to the send method will become the value of the pending yield
expression, it follows that you can only make a call like my_coro.send(42) if the coro‐
utine is currently suspended. But that’s not the case if the coroutine has never been
activated—when its state is 'GEN_CREATED'. That’s why the first activation of a coroutine
is always done with next(my_coro)—you can also call my_coro.send(None), and the
effect is the same.

In [0]:
# Example 16-2. A coroutine that yields twice

def simple_coro2(a):
  print('-> Started: a = ', a)
  b = yield a
  print('-> Received: b=', b)
  c = yield a + b
  print('-> Received: c=', c)

In [17]:
from inspect import getgeneratorstate
my_coro2 = simple_coro2(14)
getgeneratorstate(my_coro2) # inspect.getgeneratorstate reports GEN_CREATED (i.e., the coroutine has not started)

'GEN_CREATED'

In [18]:
next(my_coro2) # Advance coroutine to first yield, printing -> Started: a = 14 message then yielding value of a and suspending to wait for value to be assigned to b.

-> Started: a =  14


14

In [19]:
getgeneratorstate(my_corob2)

NameError: ignored

In [20]:
my_coro2.send(28)   # Send number 28 to suspended coroutine; the yield expression evaluates to 28 and that number is bound to b. The -> Received: b = 28 message is displayed,
                    # the value of a + b is yielded (42), and the coroutine is suspended waiting for the value to be assigned to c

-> Received: b= 28


42

In [21]:
my_coro2.send(99)  # Send number 99 to suspended coroutine; the yield expression evaluates to 99 the number is bound to c. The -> Received: c = 99 message is displayed, then
                   # the coroutine terminates, causing the generator object to raise StopIteration.

-> Received: c= 99


StopIteration: ignored

In [22]:
getgeneratorstate(my_coro2)  # getgeneratorstate reports GEN_CLOSED (i.e., the coroutine execution has completed).

'GEN_CLOSED'

### Remark

It’s crucial to understand that the execution of the coroutine is suspended exactly at the
yield keyword. As mentioned before, in an assignment statement, the code to the right
of the = is evaluated before the actual assignment happens. This means that in a line like
b = yield a, the value of b will only be set when the coroutine is activated later by the
client code. 

### Example: Coroutine to Compute a Running Average

In [0]:
# Example 16-3. coroaverager0.py: code for a running average coroutine
def averager():
  total = 0.0
  count = 0
  average = None
  while True:             # This infinite loop means this coroutine will keep on accepting values and producing results as long as the caller sends them. This coroutine will only
                          # terminate when the caller calls .close() on it, or when it’s garbage collected because there are no more references to it.
      
    term = yield average  # The yield statement here is used to suspend the coroutine, produce a result to
                          # the caller, and—later—to get a value sent by the caller to the coroutine, which resumes its infinite loop.
    total += term
    count += 1
    average = total/count

In [0]:
coro_avg = averager()
next(coro_avg)

In [26]:
coro_avg.send(10)

10.0

In [27]:
coro_avg.send(30)

20.0

In [28]:
coro_avg.send(40)

26.666666666666668

### Remark

In the doctest (Example 16-4), the call next(coro_avg) makes the coroutine advance
to the yield, yielding the initial value for average, which is None, so it does not appear
on the console. At this point, the coroutine is suspended at the yield, waiting for a value
to be sent. The line coro_avg.send(10) provides that value, causing the coroutine to
activate, assigning it to term, updating the total, count, and average variables, and
then starting another iteration in the while loop, which yields the average and waits
for another term.